#### Example reading local .nc files and plotting of 60 hz biolume raw data against other variables

Execute this in from a `jupyter notebook` command so that you may zoom into the datashader plots - this [does not work in VS Code](https://github.com/holoviz/datashader/issues/739#issuecomment-482936944).



In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("../src/data"))
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import holoviews as hv
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH, MISSIONNETCDFS

# Assumes that data have been processed locally using src/data/process_dorado.py
auv_name = "dorado"


In [2]:
# Pick mission
missions_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS)
mission_picker = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(missions_dir)) if f != ".DS_Store"]
)
display(mission_picker)


In [ ]:

netcdfs_dir = os.path.join(BASE_PATH, auv_name, MISSIONNETCDFS, mission_picker.value)
opendap_base = os.path.join("http://dods.mbari.org:8181/opendap/data/auvctd/surveys/", mission_picker.value.split('.')[0], 'netcdf')
# Use `opendap_base` to test new hyrax-docker opendap server
ds_align = xr.open_dataset(os.path.join(netcdfs_dir, f"{auv_name}_{mission_picker.value}_align.nc"))
ds_1S = xr.open_dataset(os.path.join(netcdfs_dir, f"{auv_name}_{mission_picker.value}_1S.nc"))
print(os.path.join(opendap_base, f"{auv_name}_{mission_picker.value}_align.nc"))
ds_align


In [ ]:
# Takes about 30 seconds for local netcdf files, about twice as long for opendap urls
# Do not commit to the repo the output of this cell, it is too large - about 70 MB
hv.extension("bokeh")   # See https://github.com/holoviz/holoviews/issues/4861#issuecomment-1239739743
biolume_raw_plot = ds_align["biolume_raw"].to_pandas().rename("raw").hvplot()
biolume_avg_plot = ds_align["biolume_avg_biolume"].to_pandas().rename("avg").hvplot()
biolume_avg_plot * biolume_raw_plot